In [1]:
# uncomment to install keras
# !pip install --user keras

In [2]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU,Conv1D,MaxPooling1D
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import gc
from sklearn.model_selection import train_test_split
from keras.models import load_model

Using TensorFlow backend.


In [3]:
train = pd.read_csv('./train.csv')
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [4]:
#Split into training and test set:
# X_train, X_test, y_train, y_test = train_test_split(train, train[["severe_toxicity", "obscene", "identity_attack", "insult", "threat"]], test_size = 0.10, random_state = 42)
X_train, X_test, y_train, y_test = train_test_split(train.copy()[:10000], train.copy()[:10000][["severe_toxicity", "obscene", "identity_attack", "insult", "threat"]], test_size = 0.10, random_state = 42)

In [5]:
X_train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
4896,247549,0.200000,"You put a lot of stock in the DSM, not surpris...",0.000000,0.0,0.0,0.200000,0.0,NaN,NaN,...,42251,approved,0,0,0,0,0,0.000000,0,5
4782,247415,0.000000,That's sad. Back when I graduated from Duck U...,0.000000,0.0,0.0,0.000000,0.0,NaN,NaN,...,42531,approved,0,0,0,1,0,0.000000,0,4
1496,242421,0.000000,"How did ""Civil"" choose to ""reserve"" usernames,...",0.000000,0.0,0.0,0.000000,0.0,NaN,NaN,...,36937,approved,0,0,0,0,0,0.000000,0,4
1957,243444,0.000000,<3 <3,0.000000,0.0,0.0,0.000000,0.0,NaN,NaN,...,37721,approved,0,0,0,0,0,0.000000,0,10
9171,253576,0.428571,why do you hate our inalienable rights so much...,0.014286,0.1,0.0,0.414286,0.0,NaN,NaN,...,50579,approved,0,0,0,1,0,0.071429,0,70


In [6]:
# Store the comments as seperate variables for further processing.
list_sentences_train = X_train["comment_text"]
list_sentences_test = X_test["comment_text"]

In [7]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features,char_level=True)

In [8]:
tokenizer.fit_on_texts(list(list_sentences_train))


In [9]:
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_sentences_test = tokenizer.texts_to_sequences(list_sentences_test)


In [10]:
list_tokenized_train[:1]

[[17,
  4,
  13,
  1,
  16,
  13,
  3,
  1,
  5,
  1,
  11,
  4,
  3,
  1,
  4,
  19,
  1,
  8,
  3,
  4,
  14,
  25,
  1,
  6,
  7,
  1,
  3,
  10,
  2,
  1,
  12,
  8,
  15,
  24,
  1,
  7,
  4,
  3,
  1,
  8,
  13,
  9,
  16,
  9,
  6,
  8,
  6,
  7,
  18,
  24,
  29,
  29,
  21,
  13,
  3,
  1,
  3,
  10,
  2,
  1,
  7,
  5,
  3,
  6,
  4,
  7,
  5,
  11,
  1,
  6,
  7,
  8,
  3,
  6,
  3,
  13,
  3,
  2,
  8,
  1,
  4,
  19,
  1,
  10,
  2,
  5,
  11,
  3,
  10,
  1,
  12,
  4,
  2,
  8,
  7,
  26,
  3,
  1,
  8,
  6,
  7,
  14,
  2,
  1,
  6,
  3,
  26,
  8,
  1,
  7,
  4,
  3,
  1,
  21,
  5,
  8,
  2,
  12,
  1,
  4,
  7,
  1,
  5,
  7,
  17,
  1,
  4,
  21,
  30,
  2,
  14,
  3,
  6,
  23,
  2,
  1,
  11,
  5,
  21,
  4,
  9,
  5,
  3,
  4,
  9,
  17,
  1,
  15,
  2,
  5,
  8,
  13,
  9,
  2,
  24,
  1,
  21,
  13,
  3,
  1,
  4,
  7,
  1,
  5,
  1,
  14,
  4,
  7,
  8,
  2,
  7,
  8,
  13,
  8,
  1,
  4,
  19,
  1,
  8,
  17,
  15,
  16,
  3,
  4,
  15,
  8,
  24,
  1,
  5,
 

Set length of all sentences to 500 characters. Pad zeros for sentences with length < 500

In [11]:
maxlen = 500
X_t = pad_sequences(list_tokenized_train, maxlen=maxlen)
X_te = pad_sequences(list_sentences_test, maxlen=maxlen)

In [12]:
X_t[:1]

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        17,  4, 13,  1, 16, 13,  3,  1,  5,  1, 11,  4,  3,  1,  4, 19,
         1,  8,  3,  4, 14, 25,  1,  6,  7,  1,  3, 10,  2,  1, 12,  8,
        15, 24,  1,  7,  4,  3,  1,  8, 13,  9, 16,  9,  6,  8,  6,  7,
        18, 24, 29, 29, 21, 13,  3,  1,  3, 10,  2,  1,  7,  5,  3,  6,
         4,  7,  5, 11,  1,  6,  7,  8,  3,  6,  3, 13,  3,  2, 

In [13]:
inp = Input(shape=(maxlen, ))


In [14]:
embed_size = 240
x = Embedding(len(tokenizer.word_index)+1, embed_size)(inp)

In [15]:
x = Conv1D(filters=100,kernel_size=4,padding='same', activation='relu')(x)


In [16]:
x=MaxPooling1D(pool_size=4)(x)


W1121 22:24:40.880164 139847147374400 deprecation_wrapper.py:119] From /home/dkn019/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [17]:
x = Bidirectional(GRU(60, return_sequences=True,name='lstm_layer',dropout=0.2,recurrent_dropout=0.2))(x)


In [18]:
x = GlobalMaxPool1D()(x)



In [19]:
x = Dense(50, activation="relu")(x)


In [20]:
x = Dropout(0.2)(x)
x = Dense(5, activation="sigmoid")(x)

In [21]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                 metrics=['accuracy'])

W1121 22:24:43.881327 139847147374400 deprecation.py:323] From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
model.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 240)          34080     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 100)          96100     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 125, 100)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 125, 120)          57960     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 120)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6050

In [23]:
batch_size = 32
epochs = 2

In [24]:
hist = model.fit(X_t,y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_te,y_test))

W1121 22:24:47.921477 139847147374400 deprecation_wrapper.py:119] From /home/dkn019/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 9000 samples, validate on 1000 samples
Epoch 1/2
9000/9000 [==============================] - 87s 10ms/step - loss: 0.8730 - accuracy: 0.9049 - val_loss: 0.1031 - val_accuracy: 0.9104
Epoch 2/2
9000/9000 [==============================] - 86s 10ms/step - loss: 0.1010 - accuracy: 0.9087 - val_loss: 0.0998 - val_accuracy: 0.9104


In [35]:
test_data = pd.read_csv('./test.csv')

In [36]:
list_sentences_test_data = test_data["comment_text"]


In [37]:
list_tokenized_test_data = tokenizer.texts_to_sequences(list_sentences_test_data)

In [38]:
X_test_data = pad_sequences(list_tokenized_submit, maxlen=maxlen)

In [39]:
y_test_data = model.predict(X_test_data,batch_size=batch_size,verbose=1)

97320/97320 [==============================] - 501s 5ms/step


In [41]:
df_results = pd.DataFrame(y_test_data, columns = ["severe_toxicity", "obscene", "identity_attack", "insult", "threat"]) 

In [46]:
# df_results

In [47]:
# test_data

In [48]:
df = pd.concat([test_data.reset_index(drop=True),df_results.reset_index(drop=True)], axis=1)


In [49]:
df

,id,comment_text,severe_toxicity,obscene,identity_attack,insult,threat
0,7097320,[ Integrity means that you pay your debts.]\n\...,0.007053,0.025121,0.035050,0.078540,0.013952
1,7097321,This is malfeasance by the Administrator and t...,0.008527,0.028917,0.039973,0.082723,0.016830
2,7097322,@Rmiller101 - Spoken like a true elitist. But ...,0.004193,0.016817,0.025344,0.060786,0.009188
3,7097323,"Paul: Thank you for your kind words. I do, in...",0.013280,0.037047,0.050119,0.102638,0.022696
4,7097324,Sorry you missed high school. Eisenhower sent ...,0.002204,0.010087,0.015840,0.045210,0.005193
5,7097325,Let’s see if I understand this; Berkowitz anno...,0.006476,0.021689,0.031499,0.074782,0.012031
6,7097326,Our oils read; President IS taking different ...,0.005610,0.021129,0.029849,0.069115,0.011906
7,7097327,'Work together'? Dream on. The liberals are al...,0.006021,0.022101,0.031899,0.071497,0.012386
8,7097328,What would Jerry Prevo think about that story?...,0.006395,0.023370,0.033205,0.074774,0.013011
9,7097329,When wil the indigenous be accountable and t...,0.006662,0.024371,0.034320,0.076129,0.013565
